In [8]:
!unzip /content/MP_Data.zip

Archive:  /content/MP_Data.zip
   creating: MP_Data/A/
   creating: MP_Data/A/0/
  inflating: MP_Data/A/0/0.npy       
  inflating: MP_Data/A/0/1.npy       
  inflating: MP_Data/A/0/10.npy      
  inflating: MP_Data/A/0/11.npy      
  inflating: MP_Data/A/0/12.npy      
  inflating: MP_Data/A/0/13.npy      
  inflating: MP_Data/A/0/14.npy      
  inflating: MP_Data/A/0/15.npy      
  inflating: MP_Data/A/0/16.npy      
  inflating: MP_Data/A/0/17.npy      
  inflating: MP_Data/A/0/18.npy      
  inflating: MP_Data/A/0/19.npy      
  inflating: MP_Data/A/0/2.npy       
  inflating: MP_Data/A/0/20.npy      
  inflating: MP_Data/A/0/21.npy      
  inflating: MP_Data/A/0/22.npy      
  inflating: MP_Data/A/0/23.npy      
  inflating: MP_Data/A/0/24.npy      
  inflating: MP_Data/A/0/25.npy      
  inflating: MP_Data/A/0/26.npy      
  inflating: MP_Data/A/0/27.npy      
  inflating: MP_Data/A/0/28.npy      
  inflating: MP_Data/A/0/29.npy      
  inflating: MP_Data/A/0/3.npy       
  infla

In [2]:
pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 43.8 MB/s eta 0:00:00


In [10]:

import cv2, numpy as np
import os, string
import mediapipe as mp

med_dr = mp.solutions.drawing_utils
med_dr_st = mp.solutions.drawing_styles
med_hnds = mp.solutions.hands

def medp_det_l(image, model):

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

def drw_keypoints(image, results):
    if results.mul_hand_points:
      for hand_points in results.mul_hand_points:
        med_dr.draw_landmarks(
            image,
            hand_points,
            med_hnds.HAND_CONNECTIONS,
            med_dr_st.get_default_hand_landmarks_style(),
            med_dr_st.get_default_hand_connections_style())


def extcrt_kpoints(results):
    if results.mul_hand_points:
      for hand_points in results.mul_hand_points:
        rh = np.array([[res.x, res.y, res.z] for res in hand_points.landmark]).flatten() if hand_points else np.zeros(21*3)
        return(np.concatenate([rh]))

Dataset_Path = os.path.join('MP_Data')

dig = list(np.arange(10))
alpha= list(string.ascii_uppercase)
# char = dig + alpha
# actions = np.array(list(string.ascii_uppercase))
actions = np.array(alpha)

m = 15

n = 15


In [ ]:
from func import *
from time import sleep

for action in actions:
    for sequence in range(m):
        try:
            os.makedirs(os.path.join(Dataset_Path, action, str(sequence)))
        except:
            pass


with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:

    #

    for action in actions:
        try:
            print("ACTION",action)
            for sequence in range(m):
                print("sequence",sequence)
                # looping through video squence
                for frame_num in range(n):
                    try:
                        print("Action",action,'seq',sequence,"frame_num",frame_num)
                        action_folder = os.path.join('Sign-Language-Digits-Dataset', 'Dataset', action)
                        files = os.listdir(action_folder)


                        # ret, frame = cap.read()
                        frame=cv2.imread('Sign-Language-Digits-Dataset\Dataset/{}/{}'.format(action,files[frame_num]))
                        # frame=cv2.imread('{}{}.png'.format(action,sequence))
                        # frame=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

                        image, results = medp_det_l(frame, hands)
        #                 print("results>>>>>",results)

                        drw_keypoints(image, results)

                        if frame_num == 0:
                            cv2.putText(image, 'STARTING COLLECTION', (120,200),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                            cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                            cv2.imshow('OpenCV Feed', image)
                            cv2.waitKey(200)
                        else:
                            cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                            cv2.imshow('OpenCV Feed', image)

                        k_points = extcrt_kpoints(results)
                        npy_path = os.path.join(Dataset_Path, action, str(sequence), str(frame_num))
                        np.save(npy_path, k_points)


                    except Exception as e:
                        continue
        except Exception as e:
            print('Exception',e)
            continue

    cv2.destroyAllWindows()

In [11]:
# from function import *
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import TensorBoard
label_map = {label:num for num, label in enumerate(actions)}
# print(label_map)
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

X = np.array(sequences)
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,63)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))
res = [.7, 0.2, 0.1]

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.fit(X_train, y_train, epochs=200, callbacks=[tb_callback])
model.summary()

model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save('model.h5')

KeyboardInterrupt: ignored

In [13]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import TensorBoard
label_map = {label:num for num, label in enumerate(actions)}

print(label_map)
seqs, labls = [], []

for action in actions:
    print("ACTION-->>",action,"sequence->")
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
          try:
            res = np.load(os.path.join(Dataset_Path, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
            seqs.append(window)
            labls.append(label_map[action])
          except Exception as e:
             print("ERROR=-->>>>",e)

X = np.array(seqs)
y = to_categorical(labls).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(15,63)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))
res = [.7, 0.2, 0.1]

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.fit(X_train, y_train, epochs=200, callbacks=[tb_callback])
model.summary()

model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save('model.h5')

ACTION-->> A sequence->
ACTION-->> B sequence->
ACTION-->> C sequence->
ACTION-->> D sequence->
ACTION-->> E sequence->
ACTION-->> F sequence->
ACTION-->> G sequence->
ACTION-->> H sequence->
ACTION-->> I sequence->
ACTION-->> J sequence->
ACTION-->> K sequence->
ACTION-->> L sequence->
ACTION-->> M sequence->
ACTION-->> N sequence->
ACTION-->> O sequence->
ACTION-->> P sequence->
ACTION-->> Q sequence->
ACTION-->> R sequence->
ACTION-->> S sequence->
ACTION-->> T sequence->
ACTION-->> U sequence->
ACTION-->> V sequence->
ACTION-->> W sequence->
ACTION-->> X sequence->
ACTION-->> Y sequence->
ACTION-->> Z sequence->
Epoch 1/200
174/174 [==============================] - 15s 51ms/step - loss: 1.7580 - categorical_accuracy: 0.4531
Epoch 2/200
174/174 [==============================] - 9s 52ms/step - loss: 0.4234 - categorical_accuracy: 0.8625
Epoch 3/200
174/174 [==============================] - 9s 53ms/step - loss: 0.0710 - categorical_accuracy: 0.9775
Epoch 4/200
174/174 [============

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
